## dash_oil_and_gas
#### show the use of dashapp in building the Dash Oil and Gas Gallery app

In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import dashapp
# import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import pdb
import copy
pn = dashapp.pn
pnnc = dashapp.pnnc

### First define a url_base_pathname, used later in the app definition.
The url_base_pathname is useful if you are accessing the app via something like an nginx proxy.

In [2]:
url_base_pathname = None
# url_base_pathname = '/oilgas/'
logger = dashapp.init_root_logger()

### dash oil and gas

#### Get Data

In [3]:
df_well_info = pd.read_csv('oil_gas_data/df_well_info.csv',low_memory=False)
df_well_production_by_year = pd.read_csv('oil_gas_data/df_well_production_by_year.csv')    

df_well_status = df_well_info[['Well_Status','wstatus']].drop_duplicates().sort_values('Well_Status')
df_well_status.index = list(range(len(df_well_status)))

df_well_type = df_well_info[['Well_Type','wtype']].drop_duplicates().sort_values('Well_Type')
df_well_type.index = list(range(len(df_well_type)))

df_well_color = df_well_info[['Well_Type','wcolor']].drop_duplicates().sort_values('Well_Type')
df_well_color.index = list(range(len(df_well_color)))

well_status_options = [{"label": wt[1], "value": wt[0]} for wt in df_well_status.values]
well_type_options = [{"label": wt[1], "value": wt[0]} for wt in df_well_type.values]
well_color_options = [{"label": wt[1], "value": wt[0]} for wt in df_well_color.values]


### Define a method to aggregate well data for panel and graph display

In [4]:
def get_well_aggregates(df_in,year_array,well_status_list=None,well_type_list=None):
    df_temp = df_in[(df_in.Year_Well_Completed>=year_array[0]) & (df_in.Year_Well_Completed<=year_array[1])]
    df_ptemp = df_well_production_by_year[(df_well_production_by_year.year>=year_array[0])&(df_well_production_by_year.year<=year_array[1])]
    df_ptemp = df_ptemp[df_ptemp.API_WellNo.isin(df_temp.API_WellNo.unique())]
    if well_status_list is not None:
        valid_status_ids = df1[df1.Well_Status.isin(well_status_list)].API_WellNo.values
        df_ptemp = df_ptemp[df_ptemp.API_WellNo.isin(valid_status_ids)]
    if well_type_list is not None:
        valid_type_ids = df1[df1.Well_Type.isin(well_type_list)].API_WellNo.values
        df_ptemp = df_ptemp[df_ptemp.API_WellNo.isin(valid_type_ids)]
    wells = len(df_temp.API_WellNo.unique())
    agg_data =  {
        'wells':wells,
        'gas':round(df_ptemp.gas.sum()/1000000,1),
        'water':round(df_ptemp.water.sum()/1000000,1),
        'oil':round(df_ptemp.oil.sum()/1000000,1)
    }
    return agg_data


### Define Row 1

In [5]:
def define_r1():
# Define the static components
    # ********************** plotly logo ****************************
    imgfolder = '' if url_base_pathname is None else url_base_pathname
    img_logo = html.Img(id='img_logo',src=imgfolder + "/assets/dash-logo.png",className='plogo')
    # ********************** title div ********************************
    title = html.Div(
        [html.H3("New York Oil And Gas",className='ogtitle'),
         html.H5("Production Overview",className='ogtitle')],id='title')
    # *****You ************* link to plotly info ***********************
    adiv = html.A([html.Button(["Learn More"],className='ogabutton')],id='adiv',href="https://plot.ly/dash/pricing",className='adiv')
    r1 = dashapp.multi_column_panel([img_logo,title,adiv],grid_template='1fr 4fr 1fr',
                            parent_class=None,child_class=pnnc)

    return r1


## Define Row 2

### Define Row 2 Column 1

In [6]:
def define_r2c1():
    slider = dashapp.make_slider(df_well_info,'yr_slider','Year_Well_Completed',className='oil_gas_r2_margin')
    slider_div = html.Div([
        'Filter by construction date (or select range in histogram):',
        html.P(),slider],id='slider_div')
    
    # ********************* well status radio *****************************
    df_radio_status = pd.DataFrame({'label':['All','Active only','Customize'],
                                'value':['all','active','custom']})
    radio_status = dashapp.make_radio(df_radio_status,'radio_status','value',label_column='label',
                             current_value='all')
    radio_status_div = html.Div([radio_status,'Filter by well status:',html.P()],
                      id='radio_status_div',className='oil_gas_r2_margin')

    # ********************* well status dropdown *****************************
    dropdown_status = dashapp.make_dropdown(df_well_status,'dropdown_status',
                                    'Well_Status',label_column='wstatus',
                                    current_value = df_well_status.Well_Status.unique(),
                                    multi=True)
    
    dropdown_status_div = html.Div(dropdown_status,className='oil_gas_d1_margin',id='dropdown_status_div')

    # ********************* link well status radio to dropdown *****************************
    def link_radio_status_dropdown_status_options_callback(radio_status):
        if radio_status.lower()=='all':
            ret = well_status_options
        elif radio_status.lower()=='active':
            ret = [wso for wso in well_status_options if wso['value']=='AC']
        else:
            ret = {}
        return ret
    link_radio_status_dropdown_status_options = dashapp.radio_to_dropdown_options_link(
                    radio_status,
                    dropdown_status,
                    link_radio_status_dropdown_status_options_callback)
    
    lambda_rsds_value = lambda radio_value:[wso['value'] for wso in link_radio_status_dropdown_status_options_callback(radio_value)]
    link_radio_status_dropdown_status_value_callback = lambda_rsds_value
    link_radio_status_dropdown_status_value = dashapp.radio_to_dropdown_value_link(
                    radio_status,
                    dropdown_status,
                    link_radio_status_dropdown_status_value_callback)
    
    # ********************* well type radio *****************************
    df_radio_type = pd.DataFrame({'label':['All','Productive only','Customize'],
                                'value':['all','productive','custom']})
    radio_type = dashapp.make_radio(df_radio_type,'radio_type','value',label_column='label',
                             current_value='all')
    radio_type_div = html.Div([radio_type,'Filter by well status:',html.P()],
                      id='radio_type_div',className='oil_gas_r2_margin')

    # ********************* well type dropdown *****************************
    dropdown_type = dashapp.make_dropdown(df_well_type,'dropdown_type',
                                    'Well_Type',label_column='wtype',
                                    current_value = df_well_type.Well_Type.unique(),
                                    multi=True,className='oil_gas_d1_margin')
    
    dropdown_type_div = html.Div(dropdown_type,id='dropdown_type_div')
    
    # ********************* link well status radio to dropdown *****************************
    def link_radio_type_dropdown_type_options_callback(radio_type):
        if radio_type.lower()=='all':
            ret = well_type_options
        elif radio_type.lower()=='productive':
            ret = [wto for wto in well_type_options if wto['value'] in ["GD", "GE", "GW", "IG", "IW", "OD", "OE", "OW"]]
        else:
            ret = {}
        return ret
    link_radio_type_dropdown_type_options = dashapp.radio_to_dropdown_options_link(
                    radio_type,
                    dropdown_type,
                    link_radio_type_dropdown_type_options_callback)
    
    lambda_rtdt_value = lambda radio_value:[wso['value'] for wso in link_radio_type_dropdown_type_options_callback(radio_value)]
    link_radio_type_dropdown_type_value_callback = lambda_rtdt_value
    link_radio_type_dropdown_type_value = dashapp.radio_to_dropdown_value_link(
                    radio_type,
                    dropdown_type,
                    link_radio_type_dropdown_type_value_callback)
    
#     link_radio_status_dropdown_status_options,link_radio_status_dropdown_status_value,
#     link_radio_type_dropdown_type_options,link_radio_type_dropdown_type_value

    
    divs = [slider_div,radio_status_div,dropdown_status_div,radio_type_div,dropdown_type_div]
    links = [link_radio_status_dropdown_status_options,link_radio_status_dropdown_status_value,
            link_radio_type_dropdown_type_options,link_radio_type_dropdown_type_value]
    return divs,links


### Define Row 2 Column 2


#### Define r2c2_store

In [7]:
r2c2_store = dcc.Store(id='r2c2_store',data={})
def _build_df_from_input_list(input_list,logger=None):
    year_list = input_list[0]
    year_low = int(str(year_list[0]))
    year_high = int(str(year_list[1]))
    df_temp = df_well_info[(df_well_info.Year_Well_Completed>=year_low) & (df_well_info.Year_Well_Completed<=year_high)]
    status_list = input_list[1] 
    type_list = input_list[2]
    try:
        df_temp = df_temp[df_temp.Well_Status.isin(status_list)]
        df_temp = df_temp[df_temp.Well_Type.isin(type_list)]
    except Exception as e:
        if logger is not None:
            logger.warn(f'EXCEPTION: _build_main_data_dictionary {str(e)}')
    return df_temp


def build_link_r2c2_store(input_list):
#     print(f"build_link_r2c2_store input_list: {input_list}")
    year_list = input_list[0]
    year_low = int(str(year_list[0]))
    year_high = int(str(year_list[1]))
    df_temp = _build_df_from_input_list(input_list,logger)
    df_temp2 = df_temp[["API_WellNo", "Year_Well_Completed"]].groupby(['Year_Well_Completed'],as_index=False).count()
    #{'wells': 14628, 'gas': 865.4, 'water': 15.8, 'oil': 3.8}
    aggs = get_well_aggregates(df_temp,[year_low,year_high])
    ret = {
        'data':df_temp2.to_dict('rows'),
        'year_list':year_list,
        'status_list':input_list[1],
        'type_list':input_list[2],
        'no_wells':f"{aggs['wells']} No of",
        'gas_mcf':f"{aggs['gas']}mcf",
        'oil_bbl':f"{aggs['oil']}M bbl",
        'water_bbl':f"{aggs['water']}M bbl"
    }
    return [ret]
link_r2c2_store = dashapp.DashLink(
    [('yr_slider','value'),('dropdown_status','value'),('dropdown_type','value')],
    [(r2c2_store,'data')],build_link_r2c2_store)


#### Define Row 2 Column 2 Row 1

In [8]:
def define_r2c2r1(r2c2_store):
    r2c2r1_no_wells = html.Div(id='no_wells')
    r2c2r1_gas_mcf = html.Div(id='gas_mcf')
    r2c2r1_oil_bbl = html.Div(id='oil_bbl')
    r2c2r1_water_bbl = html.Div(id='water_bbl')
    # create an array of div id's that you will use when creating DashLink instances below
    r2c2r1_panel_ids = ['no_wells','gas_mcf','oil_bbl','water_bbl']

    def build_panel_link_closure(panel_id):
        def build_panel_link(input_list):
#             print(f'build_panel_link input_list: {input_list}')
            r2c2_store = input_list[0]
            if panel_id not in r2c2_store:
                dashapp.stop_callback(f'build_panel_link no data for panel_id {panel_id} with data {input_list}',logger)        
            ret = r2c2_store[panel_id]
            return [ret]
        return build_panel_link

    # define array of links to store component, and the loop through r2c2r1_panel_ids to 
    #   create a DashLink instance for each panel (there are 4)
    r2c2r1_links = []
    for panel_id in r2c2r1_panel_ids:
        r2c2r1_link = dashapp.DashLink(
            [(r2c2_store,'data')],[(panel_id,'children')],
            build_panel_link_closure(panel_id)
        ) 
        r2c2r1_links.append(r2c2r1_link)


    r2c2r1_wells_div = html.Div([r2c2r1_no_wells,html.P('Wells')],id='wells_div')
    r2c2r1_gas_div = html.Div([r2c2r1_gas_mcf,html.P('Gas')],id='gas_div')
    r2c2r1_oil_div = html.Div([r2c2r1_oil_bbl,html.P('Oil')],id='oil_div')
    r2c2r1_water_div = html.Div([r2c2r1_water_bbl,html.P('Water')],id='water_div')
    r2c2r1_divs = [r2c2r1_wells_div,r2c2r1_gas_div,
                           r2c2r1_oil_div,r2c2r1_water_div,r2c2_store]
    return r2c2r1_divs,r2c2r1_links


#### Define Row 2 Column 2 Row 2 (*the graph of wells per year, and given the values from the dropdowns in r2c1*)

In [9]:
def define_r2c2r2(r2c2_store):
    def xygraph_fig(df_wells_per_year):
        xyfig = dashapp.plotly_plot(
            df_in=df_wells_per_year,
            x_column='Year_Well_Completed',bar_plot=True,number_of_ticks_display=10,
            plot_title='Completed Wells/Year')
        return xyfig

    df_wells_per_year = df_well_info[["API_WellNo", "Year_Well_Completed"]].groupby(['Year_Well_Completed'],as_index=False).count()
    init_fig = xygraph_fig(df_wells_per_year)
    xygraph = dcc.Graph(id='xygraph',figure=init_fig)

    # create the call back that will react to changes in r2c2_store, and modify the graph in r2c2r2
    def build_link_store_xygraph(input_list):
        r2c2_store = input_list[0]
        if 'data' not in r2c2_store:
            dashapp.stop_callback(f'build_link_store_xygraph no DataFrame for graph {input_list}',logger)        
        dict_df = r2c2_store['data']
        df_wells_per_year = dashapp.make_df(dict_df)
        xyfig = xygraph_fig(df_wells_per_year)
        return [xyfig]
    link_store_xygraph = dashapp.DashLink([(r2c2_store,'data')],[(xygraph,'figure')],build_link_store_xygraph)
    return xygraph,link_store_xygraph

#### Define Row 3  (*the maps of wells per year, and given the values from the dropdowns in r2c1*)

In [10]:
# ************* Build row 3: the map and pie charts ********************************
mapbox_access_token = "pk.eyJ1IjoiamFja2x1byIsImEiOiJjajNlcnh3MzEwMHZtMzNueGw3NWw5ZXF5In0.fk8k06T96Ml9CLGgKmk81w"
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
    title="Satellite Overview",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style="light",
        center=dict(lon=-78.05, lat=42.54),
        zoom=7,
    ),
)

def define_r3c1():

    def _create_map_from_df(df_in):
        traces = []
        for wtype, dfff in df_in.groupby("wtype"):
            trace = dict(
                type="scattermapbox",
                lon=dfff["Surface_Longitude"],
                lat=dfff["Surface_latitude"],
                text=dfff["Well_Name"],
                customdata=dfff["API_WellNo"],
                name=wtype,
                marker=dict(size=4, opacity=0.6),
            )
            traces.append(trace)

        figure = dict(data=traces, layout=layout)
        return figure


    def build_map_figure(input_list):
        r2c2_store = input_list[0]
        if ('year_list' not in r2c2_store) or ('status_list' not in r2c2_store) or ('type_list' not in r2c2_store):
            dashapp.stop_callback(f'build_link_store_xygraph insufficient data for graph {input_list}',logger)        
        yl = r2c2_store['year_list']
        sl = r2c2_store['status_list']
        tl = r2c2_store['type_list']
        original_input_list = [yl,sl,tl]
        dff = _build_df_from_input_list(original_input_list)
        fig = _create_map_from_df(dff)
        return [fig]
    init_map_figure = _create_map_from_df(df_well_info)
    mapgraph = dcc.Graph(id='mapgraph',figure=init_map_figure)
    link_mapgraph = dashapp.DashLink(
#         [(slider,'value'),(dropdown_status,'value'),(dropdown_type,'value')],
        [(r2c2_store,'data')],
        [(mapgraph,'figure')],build_map_figure)
    
    return mapgraph,link_mapgraph

def define_r3c2():
    def _create_pie_figure_from_df(dff,years):
        layout_pie = copy.deepcopy(layout)
        selected = dff["API_WellNo"].values

        year_low = years[0]
        year_high = years[1]
        aggs = get_well_aggregates(dff,[year_low,year_high])

        index = aggs['wells']
        gas = aggs['gas']
        oil = aggs['oil']
        water = aggs['water']

        # create a DataFrame that holds counts of wells by type, sorted by wtype
        df_well_count = dff[['wtype','wcolor']].sort_values("wtype").groupby(["wtype"],as_index=False).count()
        df_well_count = df_well_count.rename(columns={'wcolor':'type_count'})

        # Create a DataFrame that holds unique combinations of type and color, sorted by wtype
        df_well_color = dff[['wtype','wcolor']].drop_duplicates().sort_values('wtype')    

        data = [
            dict(
                type="pie",
                labels=["Gas", "Oil", "Water"],
                values=[gas, oil, water],
                name="Production Breakdown",
                text=[
                    "Total Gas Produced (mcf)",
                    "Total Oil Produced (bbl)",
                    "Total Water Produced (bbl)",
                ],
                hoverinfo="text+value+percent",
                textinfo="label+percent+name",
                hole=0.5,
                marker=dict(colors=["#fac1b7", "#a9bb95", "#92d8d8"]),
                domain={"x": [0, 0.45], "y": [0.2, 0.8]},
            ),
            dict(
                type="pie",
                labels=df_well_count.wtype.values,  # this works b/c it's sorted by wtype
                values=df_well_count.type_count.values, # this works b/c it's sorted by wtype
                name="Well Type Breakdown",
                hoverinfo="label+text+value+percent",
                textinfo="label+percent+name",
                hole=0.5,
                marker=dict(colors=df_well_color.wcolor.values), # this works b/c it's sorted by wtype
                domain={"x": [0.55, 1], "y": [0.2, 0.8]},
            ),
        ]
        layout_pie["title"] = "Production Summary: {} to {}".format(
            year_low, year_high
        )
        layout_pie["font"] = dict(color="#777777")
        layout_pie["legend"] = dict(
            font=dict(color="#CCCCCC", size="10"), orientation="h", bgcolor="rgba(0,0,0,0)"
        )

        figure = dict(data=data, layout=layout_pie)
        return figure

    def build_pie_figure(input_list):
        r2c2_store = input_list[0]
        if ('year_list' not in r2c2_store) or ('status_list' not in r2c2_store) or ('type_list' not in r2c2_store):
            dashapp.stop_callback(f'build_pie_figure insufficient data for graph {input_list}',logger)        
        yl = r2c2_store['year_list']
        sl = r2c2_store['status_list']
        tl = r2c2_store['type_list']
        original_input_list = [yl,sl,tl]
        dff = _build_df_from_input_list(original_input_list)
        fig = _create_pie_figure_from_df(dff,yl)
        return [fig]

    # piegraph = dgc.FigureComponent('piegraph',None,_create_pie_figure,input_tuple_list=input_component_list)
    init_years = [df_well_info.Year_Well_Completed.min(), df_well_info.Year_Well_Completed.max()]
    init_pie_figure = _create_pie_figure_from_df(df_well_info,init_years)
    piegraph = dcc.Graph(id='piegraph',figure=init_pie_figure)
    link_piegraph = dashapp.DashLink(
#         [(slider,'value'),(dropdown_status,'value'),(dropdown_type,'value')],
        [(r2c2_store,'data')],
        [(piegraph,'figure')],build_pie_figure)
    
    return piegraph,link_piegraph

def define_r3():
    mapgraph,link_mapgraph = define_r3c1()
    piegraph,link_piegraph = define_r3c2()
    chart_divs = [mapgraph,piegraph]
    chart_links = [link_mapgraph,link_piegraph]
    return chart_divs,chart_links



## Define row 4, which displays the main DataFrame

In [11]:
import importlib
importlib.reload(dashapp)
dt_title = html.Div(
        [html.P(),html.H3("All Data"),html.P()])
r4_title = dashapp.multi_column_panel([dt_title],child_class=pn,parent_class=None)
init_display_columns = ['API_WellNo','Cnty','Completion','Well_Name','wstatus','wtype',
                       'Year_Well_Completed','Month_Well_Completed']
init_display_columns = df_well_info.columns.values
df_wi_init = df_well_info[init_display_columns]#.iloc[0:20]

dt_well_info,link_for_dynamic_paging = dashapp.make_dashtable('dt_well_info',df_in=df_wi_init,filtering=True)
r4_dt = dashapp.nopanel_cell(dt_well_info)


2020-05-27 07:16:08,677 - root - DEBUG - dt_well_info entering create_dt_div
2020-05-27 07:16:08,712 - root - DEBUG - dt_well_info exiting create_dt_div


## Put all of the rows together

In [12]:
if __name__=='__main__':
    dap = dashapp.DashApp()
    # define title row
    r1 = define_r1()
    
    # define r2c1 with slider 2 radios and 2 dropdowns
    r2c1_divs,r2c1_links = define_r2c1()
    r2c1 = dashapp.multi_row_panel(r2c1_divs,child_class=pnnc,parent_class='oil_gas_r2c1')
    
    # define r2c2r1, which has the well count panels
    r2c2r1_divs,r2c2r1_links = define_r2c2r1(r2c2_store)
    r2c2r1 = dashapp.multi_column_panel(r2c2r1_divs,grid_template=' '.join(['1fr']*4),
            parent_class=None,child_class=pn)
    
    # define the graph below the count panels in r2c2r2
    xygraph,link_store_xygraph = define_r2c2r2(r2c2_store)
    r2c2r2 = dashapp.multi_column_panel([xygraph],grid_template='1fr')
    
    # combine r2c2r1 and r2c2r2
    r2c2 = dashapp.multi_row_panel([r2c2r1,r2c2r2],grid_template='1fr 6fr',child_class=None)
    
    # combine r2c1 and r2c2
    r2 = dashapp.multi_column_panel([r2c1,r2c2],grid_template='35fr 85fr',parent_class=None,child_class=None)    
    
    # define row 3, the map and pie graphs
    chart_divs,chart_links = define_r3()
    r3 = dashapp.multi_column_panel(chart_divs,grid_template='1fr 1fr',parent_class=None, child_class = None)
    
    # define row 4, which holds dashtable
    r4 = dashapp.multi_row_panel([r4_title,r4_dt],grid_template='1fr 20fr',child_class=None)
    # combine all rows
    rows = [r1,r2,r3,r4]
    # combine all links
    links = r2c1_links+r2c2r1_links+[link_r2c2_store,link_store_xygraph,link_for_dynamic_paging]+chart_links
    # merge all rows into a single panel
    all_cols = dashapp.multi_row_panel(rows,grid_template='5fr 45fr',parent_class=None,child_class=None)
    # add the links to the DashApp instance
    dap.add_links(links)
    # run the app
    dap.create_app(all_cols,app_port=8801,app_title='Dash Oil Gas')
    

2020-05-27 07:16:09,990 - root - INFO - This app will run at the URL: http://127.0.0.1:8801


 * Serving Flask app "dashapp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-05-27 07:16:10,005 - werkzeug - INFO -  * Running on http://127.0.0.1:8801/ (Press CTRL+C to quit)
2020-05-27 07:16:11,996 - werkzeug - INFO - 127.0.0.1 - - [27/May/2020 07:16:11] "GET / HTTP/1.1" 200 -
2020-05-27 07:16:12,058 - werkzeug - INFO - 127.0.0.1 - - [27/May/2020 07:16:12] "GET /assets/custom.css?m=1583299996.0 HTTP/1.1" 200 -


serving -- package: dash_renderer[1.3.0] resource: polyfill@7.8.7.min.js => location: ['/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/dash_renderer']


2020-05-27 07:16:12,066 - werkzeug - INFO - 127.0.0.1 - - [27/May/2020 07:16:12] "GET /assets/dashapp_example_3_rows.css?m=1585932241.0 HTTP/1.1" 200 -


...

_dash_table_update_paging_closure: page_current:2 page_size: 20


2020-05-27 07:16:43,656 - werkzeug - INFO - 127.0.0.1 - - [27/May/2020 07:16:43] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:1 page_size: 20


2020-05-27 07:16:45,768 - werkzeug - INFO - 127.0.0.1 - - [27/May/2020 07:16:45] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:0 page_size: 20


In [13]:
# !jupyter nbconvert --to script dash_oil_gas.ipynb